In [1]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import windows
import pywt
from IPython.display import Audio

In [2]:
HOST_SIGNAL_FILE = "input_audio.wav"
WATERMARKED_SIGNAL_FILE = "watermarked_audio.wav"
REP_CODE = True
FRAME_LENGTH = 2048
CONTROL_STRENGTH = 1000
OVERLAP = .5
NUM_REPS = 3
WAVELET_BASIS = "db4"
WAVELET_LEVEL = 3
WAVELET_MODE = "symmetric"
THRESHOLD = .0

In [3]:
# equivalent of MATLAB 'fix' function
def fix(xs):
    if xs >= 0:
        res = np.floor(xs)
    else:
        res = np.ceil(xs)
    return res

In [4]:
def embed_watermark():
    sr, host_signal = wavfile.read(HOST_SIGNAL_FILE)
    signal_len = len(host_signal)
    frame_shift = int(FRAME_LENGTH * (1 - OVERLAP))
    overlap_length = int(FRAME_LENGTH * OVERLAP)
    embed_nbit = fix((signal_len - overlap_length) / frame_shift)

    if REP_CODE:
        effective_nbit = np.floor(embed_nbit / NUM_REPS)
        embed_nbit = effective_nbit * NUM_REPS
    else:
        effective_nbit = embed_nbit

    frame_shift = int(frame_shift)
    effective_nbit = int(effective_nbit)
    embed_nbit = int(embed_nbit)

    # this is a custom binding from text to its binary sequence
    # it serves as a non-symmetric key between the sender and the receiver
    # we randomly generate a sequence of 0s and 1s for the watermark
    # but any type of tokens can be mapped to a sequence of 0s and 1s 
    watermark_dict = {"sample_text" : np.random.randint(2, size=int(effective_nbit))}
    wmark_original = watermark_dict["sample_text"]

    if REP_CODE:
        wmark_extended = np.repeat(wmark_original, NUM_REPS)
    else:
        wmark_extended = wmark_original

    pointer = 0
    count = 0
    wmed_signal = np.zeros((frame_shift * embed_nbit))
    prev = np.zeros((FRAME_LENGTH))
    for i in range(embed_nbit):
        frame = host_signal[pointer : pointer + FRAME_LENGTH]

        coeffs = pywt.wavedec(
            data=frame, wavelet=WAVELET_BASIS, level=WAVELET_LEVEL, mode=WAVELET_MODE
        )

        # adaptive alpha
        alpha = 10 ** (int(np.log10(np.abs(np.mean(coeffs[0])))) + 1)
        
        # manual alpha
        # alpha = CONTROL_STRENGTH

        # watermark is embedded in the low frequencies
        # you can change the index to -1 for embedding in the high frequencies
        if wmark_extended[count] == 1:
            coeffs[0] = coeffs[0] - np.mean(coeffs[0]) + alpha
        else:
            coeffs[0] = coeffs[0] - np.mean(coeffs[0]) - alpha

        # reconstruct the signal from its wavelet coeffs
        wmarked_frame = pywt.waverec(
            coeffs=coeffs, wavelet=WAVELET_BASIS, mode=WAVELET_MODE
        )

        # Hann window
        wmarked_frame = wmarked_frame * windows.hann(FRAME_LENGTH)
        wmed_signal[frame_shift * i : frame_shift * (i + 1)] = np.concatenate(
            (
                prev[frame_shift:FRAME_LENGTH] + wmarked_frame[0:overlap_length],
                wmarked_frame[overlap_length:frame_shift],
            )
        )

        prev = wmarked_frame
        count = count + 1
        pointer = pointer + frame_shift

    wmed_signal = np.concatenate(
        (wmed_signal, host_signal[len(wmed_signal) : signal_len])
    )

    # some normalizations for the amplitude
    wmed_signal += np.mean(host_signal) - np.mean(wmed_signal)
    wmed_signal /= np.max(wmed_signal)
    wmed_signal *= np.max(host_signal) * 100

    wavfile.write(WATERMARKED_SIGNAL_FILE, sr, wmed_signal.astype(np.int16))

    return wmark_original

In [5]:
def extract_watermark(eval_signal=None):
    if eval_signal is None:
        _, eval_signal = wavfile.read(WATERMARKED_SIGNAL_FILE)
    signal_len = len(eval_signal)
    frame_shift = int(FRAME_LENGTH * (1 - OVERLAP))
    embed_nbit = fix((signal_len - FRAME_LENGTH * OVERLAP) / frame_shift)

    if REP_CODE:
        effective_nbit = np.floor(embed_nbit / NUM_REPS)
        embed_nbit = effective_nbit * NUM_REPS
    else:
        effective_nbit = embed_nbit

    frame_shift = int(frame_shift)
    effective_nbit = int(effective_nbit)
    embed_nbit = int(embed_nbit)

    pointer = 0
    detected_bit = np.zeros(embed_nbit)
    for i in range(embed_nbit):
        wmarked_frame = eval_signal[pointer : pointer + FRAME_LENGTH]

        # wavelet decomposition
        wmarked_coeffs = pywt.wavedec(
            data=wmarked_frame,
            wavelet=WAVELET_BASIS,
            level=WAVELET_LEVEL,
            mode=WAVELET_MODE,
        )

        if np.sum(wmarked_coeffs[0]) >= THRESHOLD:
            detected_bit[i] = 1
        else:
            detected_bit[i] = 0

        pointer = pointer + frame_shift

    if REP_CODE:
        count = 0
        wmark_recovered = np.zeros(effective_nbit)

        for i in range(effective_nbit):
            ave = np.sum(detected_bit[count : count + NUM_REPS]) / NUM_REPS
            if ave >= 0.5:
                wmark_recovered[i] = 1
            else:
                wmark_recovered[i] = 0

            count = count + NUM_REPS
    else:
        wmark_recovered = detected_bit

    return wmark_recovered

In [6]:
wmark_original = embed_watermark()
wmark_recovered = extract_watermark()

# if the difference is zero then the signal is authentic
wmark_diff = int(np.sum(np.abs(wmark_recovered - wmark_original)))
print(f'difference between the original and the recovered watermark is {wmark_diff}')
print('\nwatermarked audio file:')
display(Audio(WATERMARKED_SIGNAL_FILE))

difference between the original and the recovered watermark is 0

watermarked audio file:


## Noise Robustness Of The Algorithm

In [7]:
_, wmed_signal = wavfile.read(WATERMARKED_SIGNAL_FILE)
max_amp = np.max(wmed_signal)
for noise_scale in np.arange(0, 20 * max_amp, max_amp):
    noisy_signal = wmed_signal + np.random.normal(0, noise_scale, len(wmed_signal))
    wmark_recovered = extract_watermark(noisy_signal)
    wmark_diff = int(np.sum(np.abs(wmark_recovered - wmark_original)))
    print(f"noise with scale {noise_scale} caused watermark diff of {wmark_diff}")

noise with scale 0 caused watermark diff of 0
noise with scale 25500 caused watermark diff of 0
noise with scale 51000 caused watermark diff of 0
noise with scale 76500 caused watermark diff of 0
noise with scale 102000 caused watermark diff of 0
noise with scale 127500 caused watermark diff of 0
noise with scale 153000 caused watermark diff of 0
noise with scale 178500 caused watermark diff of 1
noise with scale 204000 caused watermark diff of 0
noise with scale 229500 caused watermark diff of 1
noise with scale 255000 caused watermark diff of 1
noise with scale 280500 caused watermark diff of 1
noise with scale 306000 caused watermark diff of 4
noise with scale 331500 caused watermark diff of 2
noise with scale 357000 caused watermark diff of 2
noise with scale 382500 caused watermark diff of 0
noise with scale 408000 caused watermark diff of 3
noise with scale 433500 caused watermark diff of 2
noise with scale 459000 caused watermark diff of 8
noise with scale 484500 caused watermar